In [2]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim

import dlc_practical_prologue as prologue

if torch.cuda.is_available():  
  dev="cuda:0"
  device = torch.device(dev) 

In [3]:
pairs = prologue.generate_pair_sets(1000)

train_input=pairs[0].to(dev)
train_target=pairs[1].to(dev)
train_classes=pairs[2].to(dev)
test_input=pairs[3].to(dev)
test_target=pairs[4].to(dev)
test_classes=pairs[5].to(dev)

train_input=train_input.view((1000,-1)) #Vector convertion, images are concatenated
test_input=test_input.view((1000,-1))

In [27]:
# t=torch.tensor([[[[1,2,3],[4,5,6],[7,8,9]],[[10,11,12],[13,14,15],[16,17,18]]],[[[1,2,3],[4,5,6],[7,8,9]],[[10,11,12],[13,14,15],[16,17,18]]]])
# t.view(2,-1)

tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18]])

In [5]:

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(392, 500)
        self.fc2 = nn.Linear(500, 20)
        self.fc3 = nn.Linear(20, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
def train_model(model, train_input, train_target):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr = 1e-3)
    nb_epochs = 100
    mini_batch_size= 100

    for e in range(nb_epochs):
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [7]:
def compute_nb_errors(model, data_input, data_target):

    nb_data_errors = 0
    mini_batch_size= 100
    for b in range(0, data_input.size(0), mini_batch_size):
        output = model(data_input.narrow(0, b, mini_batch_size))
        _, predicted_classes = torch.max(output, 1)
        for k in range(mini_batch_size):
            if data_target[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1

    return nb_data_errors

In [9]:
train_E=[]
test_E=[]

for k in range(5):
    model=Net().cuda()
    train_model(model,train_input,train_target)
    train_E.append(compute_nb_errors(model, train_input, train_target) / train_input.size(0) * 100)
    test_E.append(compute_nb_errors(model, test_input, test_target) / test_input.size(0) * 100)
    print('train_error {:.02f}% test_error {:.02f}%'.format(train_E[-1],test_E[-1]))

train_error 0.30% test_error 22.80%
train_error 0.20% test_error 21.30%
train_error 0.00% test_error 21.60%
train_error 0.40% test_error 24.00%
train_error 0.90% test_error 24.70%
